In [1]:
import random
import argparse
import numpy as np

from preprocessing import parse_annotation
import json

argparser = argparse.ArgumentParser()

argparser.add_argument(
    '-c',
    '--conf',
    default='config.json',
    help='path to configuration file')

argparser.add_argument(
    '-a',
    '--anchors',
    default=5,
    help='number of anchors to use')

def IOU(ann, centroids):
    w, h = ann
    similarities = []

    for centroid in centroids:
        c_w, c_h = centroid

        if c_w >= w and c_h >= h:
            similarity = w*h/(c_w*c_h)
        elif c_w >= w and c_h <= h:
            similarity = w*c_h/(w*h + (c_w-w)*c_h)
        elif c_w <= w and c_h >= h:
            similarity = c_w*h/(w*h + c_w*(c_h-h))
        else: #means both w,h are bigger than c_w and c_h respectively
            similarity = (c_w*c_h)/(w*h)
        similarities.append(similarity) # will become (k,) shape

    return np.array(similarities)

def avg_IOU(anns, centroids):
    n,d = anns.shape
    sum = 0.

    for i in range(anns.shape[0]):
        sum+= max(IOU(anns[i], centroids))

    return sum/n

def print_anchors(centroids):
    anchors = centroids.copy()

    widths = anchors[:, 0]
    sorted_indices = np.argsort(widths)

    r = "anchors: ["
    for i in sorted_indices[:-1]:
        r += '%0.2f,%0.2f, ' % (anchors[i,0], anchors[i,1])

    #there should not be comma after last anchor, that's why
    r += '%0.2f,%0.2f' % (anchors[sorted_indices[-1:],0], anchors[sorted_indices[-1:],1])
    r += "]"

    print(r)
    return r

def run_kmeans(ann_dims, anchor_num):
    ann_num = ann_dims.shape[0]
    iterations = 0
    prev_assignments = np.ones(ann_num)*(-1)
    iteration = 0
    old_distances = np.zeros((ann_num, anchor_num))

    indices = [random.randrange(ann_dims.shape[0]) for i in range(anchor_num)]
    centroids = ann_dims[indices]
    anchor_dim = ann_dims.shape[1]

    while True:
        distances = []
        iteration += 1
        for i in range(ann_num):
            d = 1 - IOU(ann_dims[i], centroids)
            distances.append(d)
        distances = np.array(distances) # distances.shape = (ann_num, anchor_num)

        print("iteration {}: dists = {}".format(iteration, np.sum(np.abs(old_distances-distances))))

        #assign samples to centroids
        assignments = np.argmin(distances,axis=1)

        if (assignments == prev_assignments).all() :
            return centroids

        #calculate new centroids
        centroid_sums=np.zeros((anchor_num, anchor_dim), np.float)
        for i in range(ann_num):
            centroid_sums[assignments[i]]+=ann_dims[i]
        for j in range(anchor_num):
            centroids[j] = centroid_sums[j]/(np.sum(assignments==j) + 1e-6)

        prev_assignments = assignments.copy()
        old_distances = distances.copy()


Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/ankish1/anaconda3/envs/tensor/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/ankish1/anaconda3/envs/tensor/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/ankish1/anaconda3/envs/tensor/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
ImportError: dlopen: cannot load any more object with static TLS


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.

In [10]:

def main():
    config_path = "/home/ankish/kaggle_data_science/yolo3/retinaNet/keras-yolo2/config.json"#args.conf
    num_anchors = 10#args.anchors

    with open(config_path) as config_buffer:
        config = json.loads(config_buffer.read())

    train_imgs, train_labels = parse_annotation(config['train']['train_annot_folder'],
                                                config['train']['train_image_folder'],
                                                config['model']['labels'])

    grid_w = config['model']['input_size']/32
    grid_h = config['model']['input_size']/32

    # run k_mean to find the anchors
    annotation_dims = []
    for image in train_imgs:
        cell_w = image['width']/grid_w
        cell_h = image['height']/grid_h

        for obj in image['object']:
            relative_w = (float(obj['xmax']) - float(obj['xmin']))/cell_w
            relatice_h = (float(obj["ymax"]) - float(obj['ymin']))/cell_h
            annotation_dims.append(tuple(map(float, (relative_w,relatice_h))))

    annotation_dims = np.array(annotation_dims)
    print("annotation_dims: ", annotation_dims)
    centroids = run_kmeans(annotation_dims, num_anchors)

    # write anchors to file
    print('\naverage IOU for', num_anchors, 'anchors:', '%0.2f' % avg_IOU(annotation_dims, centroids))
    print_anchors(centroids)

# if __name__ == '__main__':
#     args = argparser.parse_args()
#     main(args)


In [115]:
config_path = "/home/ankish/kaggle_data_science/yolo3/retinaNet/keras-yolo2/config.json"#args.conf
num_anchors = 10#args.anchors

with open(config_path) as config_buffer:
    config = json.loads(config_buffer.read())

# train_imgs, train_labels = parse_annotation(config['train']['train_annot_folder'],
#                                             config['train']['train_image_folder'],
#                                             config['model']['labels'])
img_dir = '/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/images/'
ann_dir = '/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/annotations/'
train_imgs, train_labels = parse_annotation(ann_dir, img_dir)

grid_w = config['model']['input_size']/32
grid_h = config['model']['input_size']/32

# run k_mean to find the anchors
annotation_dims = []
for image in train_imgs:
    cell_w = image['width']/grid_w
    cell_h = image['height']/grid_h
    
    count = 0
    for obj in image['object']:
        relative_w = (float(obj['xmax']) - float(obj['xmin']))/cell_w
        relatice_h = (float(obj["ymax"]) - float(obj['ymin']))/cell_h
#         print(map(float, (relative_w,relatice_h)))
#         print(tuple(map(float, (relative_w,relatice_h))))
        annotation_dims.append(tuple(map(float, (relative_w,relatice_h))))
#         break
        count += 1
    if count > 1:
        print(image)
        print(count)
        print("=====")
#     break
annotation_dims = np.array(annotation_dims)

{'object': [{'name': 'raccoon', 'xmin': 100, 'ymin': 124, 'xmax': 266, 'ymax': 324}, {'name': 'raccoon', 'xmin': 342, 'ymin': 101, 'xmax': 570, 'ymax': 297}], 'filename': '/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/images/raccoon-117.jpg', 'width': 640, 'height': 448}
2
=====
{'object': [{'name': 'raccoon', 'xmin': 16, 'ymin': 62, 'xmax': 362, 'ymax': 353}, {'name': 'raccoon', 'xmin': 211, 'ymin': 359, 'xmax': 277, 'ymax': 402}, {'name': 'raccoon', 'xmin': 198, 'ymin': 392, 'xmax': 280, 'ymax': 473}], 'filename': '/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/images/raccoon-119.jpg', 'width': 400, 'height': 533}
3
=====
{'object': [{'name': 'raccoon', 'xmin': 28, 'ymin': 21, 'xmax': 126, 'ymax': 181}, {'name': 'raccoon', 'xmin': 85, 'ymin': 33, 'xmax': 235, 'ymax': 193}], 'filename': '/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/images/raccoon-12.jpg', 'width': 259, 'height': 194}
2
=====
{'object': [{'name': 'raccoon', 'xmin

In [117]:
# print("annotation_dims: ", annotation_dims)
centroids = run_kmeans(annotation_dims, num_anchors)

# write anchors to file
print('\naverage IOU for', num_anchors, 'anchors:', '%0.2f' % avg_IOU(annotation_dims, centroids))
print_anchors(centroids)

iteration 1: dists = 1094.1701425230578
iteration 2: dists = 95.89416794976658
iteration 3: dists = 32.07709807236874
iteration 4: dists = 20.83417006580313
iteration 5: dists = 24.243086241055863
iteration 6: dists = 32.33465601254913
iteration 7: dists = 17.51010394986301
iteration 8: dists = 7.707568520988861
iteration 9: dists = 6.945878415743149
iteration 10: dists = 7.349986317484446
iteration 11: dists = 4.101095988896969
iteration 12: dists = 7.470390345297306
iteration 13: dists = 4.446576130959513
iteration 14: dists = 5.122952249932992
iteration 15: dists = 3.39908093372861
iteration 16: dists = 0.9217926706395791
iteration 17: dists = 0.9309140894985454

average IOU for 10 anchors: 0.87
anchors: [2.40,1.51, 3.54,5.08, 4.42,8.37, 4.76,5.34, 5.90,10.70, 6.85,7.26, 7.48,11.40, 9.23,11.86, 10.27,8.97, 11.60,12.02]


In [122]:
anchors = print_anchors(centroids)

anchors: [2.40,1.51, 3.54,5.08, 4.42,8.37, 4.76,5.34, 5.90,10.70, 6.85,7.26, 7.48,11.40, 9.23,11.86, 10.27,8.97, 11.60,12.02]


In [128]:
anc = [2.40,1.51, 3.54,5.08, 4.42,8.37, 4.76,5.34, 5.90,10.70, 6.85,7.26, 7.48,11.40, 9.23,11.86, 10.27,8.97, 11.60,12.02]
sorted(anc, reverse=True)

[12.02,
 11.86,
 11.6,
 11.4,
 10.7,
 10.27,
 9.23,
 8.97,
 8.37,
 7.48,
 7.26,
 6.85,
 5.9,
 5.34,
 5.08,
 4.76,
 4.42,
 3.54,
 2.4,
 1.51]

In [108]:
annotation_dims.shape, grid_h, grid_w, 416/32

((217, 2), 13.0, 13.0, 13.0)

In [18]:
config['train']['train_annot_folder']#['train_image_folder']

'/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/annotations/'

In [ ]:
for ann in sorted(os.listdir(ann_dir)):

In [25]:
import xml.etree.ElementTree as ET

tree = ET.parse('/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/annotations/raccoon-85.xml')
for elem in tree.iter():
    print(elem)

<Element 'annotation' at 0x7ff3f806f778>
<Element 'folder' at 0x7ff3f806f7c8>
<Element 'filename' at 0x7ff3f806f818>
<Element 'path' at 0x7ff3f806f868>
<Element 'source' at 0x7ff3f806f8b8>
<Element 'database' at 0x7ff3f806f908>
<Element 'size' at 0x7ff3f806f958>
<Element 'width' at 0x7ff3f806f9a8>
<Element 'height' at 0x7ff3f806f9f8>
<Element 'depth' at 0x7ff3f806fa48>
<Element 'segmented' at 0x7ff3f806fa98>
<Element 'object' at 0x7ff3f806fae8>
<Element 'name' at 0x7ff3f806fb38>
<Element 'pose' at 0x7ff3f806fb88>
<Element 'truncated' at 0x7ff3f806fbd8>
<Element 'difficult' at 0x7ff3f806fc28>
<Element 'bndbox' at 0x7ff3f806fc78>
<Element 'xmin' at 0x7ff3f806fcc8>
<Element 'ymin' at 0x7ff3f806fd18>
<Element 'xmax' at 0x7ff3f806fd68>
<Element 'ymax' at 0x7ff3f806fdb8>


In [31]:
def parse_annotation(ann_dir, img_dir, labels=[]):
    all_imgs = []
    seen_labels = {}
    
    for ann in sorted(os.listdir(ann_dir)):
        img = {'object':[]}

        tree = ET.parse(ann_dir + ann)
        
        for elem in tree.iter():
            if 'filename' in elem.tag:
                img['filename'] = img_dir + elem.text
            if 'width' in elem.tag:
                img['width'] = int(elem.text)
            if 'height' in elem.tag:
                img['height'] = int(elem.text)
            if 'object' in elem.tag or 'part' in elem.tag:
                obj = {}
                
                for attr in list(elem):
                    if 'name' in attr.tag:
                        obj['name'] = attr.text

                        if obj['name'] in seen_labels:
                            seen_labels[obj['name']] += 1
                        else:
                            seen_labels[obj['name']] = 1
                        
                        if len(labels) > 0 and obj['name'] not in labels:
                            break
                        else:
                            img['object'] += [obj]
                            
                    if 'bndbox' in attr.tag:
                        for dim in list(attr):
                            if 'xmin' in dim.tag:
                                obj['xmin'] = int(round(float(dim.text)))
                            if 'ymin' in dim.tag:
                                obj['ymin'] = int(round(float(dim.text)))
                            if 'xmax' in dim.tag:
                                obj['xmax'] = int(round(float(dim.text)))
                            if 'ymax' in dim.tag:
                                obj['ymax'] = int(round(float(dim.text)))

        if len(img['object']) > 0:
            all_imgs += [img]
                        
    return all_imgs, seen_labels

In [30]:
img_dir = '/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/images'
tree = ET.parse('/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/annotations/raccoon-85.xml')
# for elem in tree.iter():
#     print(elem)
#     if 'filename' in elem.tag:
#         print(elem.text)
#         img['filename'] = img_dir + elem.text
    
img = {'object':[]}

for elem in tree.iter():
    if 'filename' in elem.tag:
        print(img_dir + elem.text)
        img['filename'] = img_dir + elem.text
    if 'width' in elem.tag:
        print(int(elem.text))
        img['width'] = int(elem.text)
    if 'height' in elem.tag:
        print(int(elem.text))
        img['height'] = int(elem.text)

/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/imagesraccoon-85.jpg
620
465


In [33]:
img_dir = '/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/images/'
ann_dir = '/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/annotations/'
imgs, labels = parse_annotation(ann_dir, img_dir)

In [37]:
imgs[0]

{'object': [{'name': 'raccoon',
   'xmin': 81,
   'ymin': 88,
   'xmax': 522,
   'ymax': 408}],
 'filename': '/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/images/raccoon-1.jpg',
 'width': 650,
 'height': 417}

In [38]:
imgs[1]

{'object': [{'name': 'raccoon',
   'xmin': 130,
   'ymin': 2,
   'xmax': 446,
   'ymax': 488}],
 'filename': '/home/ankish/kaggle_data_science/yolo3/retinaNet/raccoon_dataset/images/raccoon-10.jpg',
 'width': 450,
 'height': 495}

In [41]:
json_dumps = json.dumps(imgs)

In [99]:
import pandas as pd

df = pd.read_json(json_dumps)
df.head()

,filename,height,object,width
0,/home/ankish/kaggle_data_science/yolo3/retinaN...,417,"[{'name': 'raccoon', 'xmin': 81, 'ymin': 88, '...",650
1,/home/ankish/kaggle_data_science/yolo3/retinaN...,495,"[{'name': 'raccoon', 'xmin': 130, 'ymin': 2, '...",450
2,/home/ankish/kaggle_data_science/yolo3/retinaN...,576,"[{'name': 'raccoon', 'xmin': 548, 'ymin': 10, ...",960
3,/home/ankish/kaggle_data_science/yolo3/retinaN...,426,"[{'name': 'raccoon', 'xmin': 86, 'ymin': 53, '...",640
4,/home/ankish/kaggle_data_science/yolo3/retinaN...,194,"[{'name': 'raccoon', 'xmin': 1, 'ymin': 1, 'xm...",259


In [100]:
df['object'][0]

[{'name': 'raccoon', 'xmin': 81, 'ymin': 88, 'xmax': 522, 'ymax': 408}]

In [101]:
xmins, ymins, xmaxs, ymaxs, names = [], [], [], [], []
for neww in df['object']:
    xmins.append(neww[0]['xmin'])
    ymins.append(neww[0]['ymin'])
    xmaxs.append(neww[0]['xmax'])
    ymaxs.append(neww[0]['ymax'])
    names.append(neww[0]['name'])
    

In [102]:
df = pd.concat([df,pd.Series(data=xmins,name='xmin')],axis=1)
df = pd.concat([df,pd.Series(data=xmaxs,name='xmax')],axis=1)
df = pd.concat([df,pd.Series(data=ymins,name='ymin')],axis=1)
df = pd.concat([df,pd.Series(data=ymaxs,name='ymax')],axis=1)
df = pd.concat([df,pd.Series(data=names,name='name')],axis=1)

df.drop(['object'], axis=1, inplace=True)
df.head()

,filename,height,width,xmin,xmax,ymin,ymax,name
0,/home/ankish/kaggle_data_science/yolo3/retinaN...,417,650,81,522,88,408,raccoon
1,/home/ankish/kaggle_data_science/yolo3/retinaN...,495,450,130,446,2,488,raccoon
2,/home/ankish/kaggle_data_science/yolo3/retinaN...,576,960,548,954,10,520,raccoon
3,/home/ankish/kaggle_data_science/yolo3/retinaN...,426,640,86,400,53,356,raccoon
4,/home/ankish/kaggle_data_science/yolo3/retinaN...,194,259,1,118,1,152,raccoon


In [136]:
from keras.models import Model
import tensorflow as tf
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.merge import concatenate
from keras.applications.mobilenet import MobileNet
from keras.applications import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50

FULL_YOLO_BACKEND_PATH  = "full_yolo_backend.h5"   # should be hosted on a server
TINY_YOLO_BACKEND_PATH  = "tiny_yolo_backend.h5"   # should be hosted on a server
SQUEEZENET_BACKEND_PATH = "squeezenet_backend.h5"  # should be hosted on a server
MOBILENET_BACKEND_PATH  = "mobilenet_backend.h5"   # should be hosted on a server
INCEPTION3_BACKEND_PATH = "inception_backend.h5"   # should be hosted on a server
VGG16_BACKEND_PATH      = "vgg16_backend.h5"       # should be hosted on a server
RESNET50_BACKEND_PATH   = "resnet50_backend.h5"    # should be hosted on a server

class BaseFeatureExtractor(object):
    """docstring for ClassName"""

    # to be defined in each subclass
    def __init__(self, input_size):
        raise NotImplementedError("error message")

    # to be defined in each subclass
    def normalize(self, image):
        raise NotImplementedError("error message")       

    def get_output_shape(self):
        return self.feature_extractor.get_output_shape_at(-1)[1:3]

    def extract(self, input_image):
        return self.feature_extractor(input_image)

    
class MobileNetFeature(BaseFeatureExtractor):
    """docstring for ClassName"""
    def __init__(self, input_size):
        input_image = Input(shape=(input_size, input_size, 3))

        mobilenet = MobileNet(input_shape=(224,224,3), include_top=False, weights=None)
        mobilenet.load_weights('mobile-net-yolo.h5')

        x = mobilenet(input_image)

        self.feature_extractor = Model(input_image, x)  

    def normalize(self, image):
        image = image / 255.
        image = image - 0.5
        image = image * 2.

        return image
    

In [145]:
from keras.models import Model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU
import tensorflow as tf
import numpy as np
import os
import cv2
from utils import decode_netout, compute_overlap, compute_ap
from keras.applications.mobilenet import MobileNet
from keras.layers.merge import concatenate
from keras.optimizers import SGD, Adam, RMSprop
from preprocessing import BatchGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
# from backend import MobileNetFeature 

input_size = 416
max_box_per_image = 10
nb_box = 5
nb_class = 10


input_image     = Input(shape=(input_size, input_size, 3))
true_boxes = Input(shape=(1, 1, 1, max_box_per_image , 4))  

feature_extractor = MobileNetFeature(input_size)

print(feature_extractor.get_output_shape())    
grid_h, grid_w = feature_extractor.get_output_shape()        
features = feature_extractor.extract(input_image)            

# make the object detection layer
output = Conv2D(nb_box * (4 + 1 + nb_class), 
                (1,1), strides=(1,1), 
                padding='same', 
                name='DetectionLayer', 
                kernel_initializer='lecun_normal')(features)
output = Reshape((grid_h, grid_w, nb_box, 4 + 1 + nb_class))(output)
output = Lambda(lambda args: args[0])([output, true_boxes])

model = Model([input_image, true_boxes], output)


# initialize the weights of the detection layer
layer = model.layers[-4]
weights = layer.get_weights()

new_kernel = np.random.normal(size=weights[0].shape)/(grid_h*grid_w)
new_bias   = np.random.normal(size=weights[1].shape)/(grid_h*grid_w)

layer.set_weights([new_kernel, new_bias])

# print a summary of the whole model
model.summary()

(13, 13)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 13, 13, 1024) 3228864     input_13[0][0]                   
__________________________________________________________________________________________________
DetectionLayer (Conv2D)         (None, 13, 13, 75)   76875       model_2[1][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 13, 13, 5, 15 0           DetectionLayer[0][0]             
__________________________________________________________________________________________________
i

In [147]:
model.layers[-4]

In [144]:

input_image     = Input(shape=(input_size, input_size, 3))
true_boxes = Input(shape=(1, 1, 1, max_box_per_image , 4))  

feature_extractor = MobileNetFeature(input_size)

feature_extractor.feature_extractor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 416, 416, 3)       0         
_________________________________________________________________
mobilenet_1.00_224 (Model)   multiple                  3228864   
Total params: 3,228,864
Trainable params: 3,206,976
Non-trainable params: 21,888
_________________________________________________________________


In [1]:
import tensorflow as tf
tfe = tf.contrib.eager
tf.enable_eager_execution()

import numpy as np

In [5]:
y_pred = np.random.randn(1,5,5,2,5) # (batch, grid_x, grid_y, n_box, n_dims)
y_true = np.where(y_pred>0.1,1,0)


In [7]:
y_true = tf.constant(y_true)
y_pred = tf.constant(y_pred)


mask_shape = tf.shape(y_true)[:4]

cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(5), [5]), (1, 5, 5, 1, 1)))
cell_y = tf.transpose(cell_x, (0,2,1,3,4))

cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [1, 1, 1, 2, 1])

coord_mask = tf.zeros(mask_shape)
conf_mask  = tf.zeros(mask_shape)
class_mask = tf.zeros(mask_shape)

seen = tfe.Variable(0.)
total_recall = tfe.Variable(0.)

In [29]:
tf.reshape(tf.concat([cell_x,cell_y], -1),(5,5,2))[:,:,0]

<tf.Tensor: id=173, shape=(5, 5), dtype=float32, numpy=
array([[ 0.,  1.,  2.,  3.,  4.],
       [ 0.,  1.,  2.,  3.,  4.],
       [ 0.,  1.,  2.,  3.,  4.],
       [ 0.,  1.,  2.,  3.,  4.],
       [ 0.,  1.,  2.,  3.,  4.]], dtype=float32)>

In [28]:
tf.reshape(tf.concat([cell_x,cell_y], -1),(5,5,2))[:,:,1]

<tf.Tensor: id=164, shape=(5, 5), dtype=float32, numpy=
array([[ 0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 2.,  2.,  2.,  2.,  2.],
       [ 3.,  3.,  3.,  3.,  3.],
       [ 4.,  4.,  4.,  4.,  4.]], dtype=float32)>

In [34]:
"""
Adjust prediction
"""
### adjust x and y      
pred_box_xy = tf.cast(tf.sigmoid(y_pred[..., :2]),'float32') + cell_grid

### adjust w and h
pred_box_wh = tf.exp(y_pred[..., 2:4])# * np.reshape(anchors, [1,1,1,2,2])

### adjust confidence
pred_box_conf = tf.sigmoid(y_pred[..., 4])

### adjust class probabilities
pred_box_class = y_pred[..., 5:]

In [40]:
pred_box_class.get_shape(), pred_box_conf.get_shape(), pred_box_wh.get_shape(), pred_box_xy.get_shape()

(TensorShape([Dimension(1), Dimension(5), Dimension(5), Dimension(2), Dimension(0)]),
 TensorShape([Dimension(1), Dimension(5), Dimension(5), Dimension(2)]),
 TensorShape([Dimension(1), Dimension(5), Dimension(5), Dimension(2), Dimension(2)]),
 TensorShape([Dimension(1), Dimension(5), Dimension(5), Dimension(2), Dimension(2)]))

In [56]:
"""
Adjust ground truth
"""
### adjust x and y
true_box_xy = y_true[..., 0:2] # relative position to the containing cell

### adjust w and h
true_box_wh = y_true[..., 2:4] # number of cells accross, horizontally and vertically

### adjust confidence
true_wh_half = true_box_wh / 2.
true_mins    = tf.cast(true_box_xy,'float64') - true_wh_half
true_maxes   = tf.cast(true_box_xy,'float64') + true_wh_half

pred_wh_half = pred_box_wh / 2.
pred_mins    = tf.cast(pred_box_xy,'float64') - pred_wh_half
pred_maxes   = tf.cast(pred_box_xy,'float64') + pred_wh_half       

intersect_mins  = tf.maximum(pred_mins,  true_mins)
intersect_maxes = tf.minimum(pred_maxes, true_maxes)
intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]

true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

union_areas = pred_areas + tf.cast(true_areas,'float64') - intersect_areas
iou_scores  = tf.truediv(intersect_areas, union_areas)

true_box_conf = iou_scores * tf.cast(y_true[..., 4],'float64')

### adjust class probabilities
# true_box_class = tf.argmax(tf.cast(y_true[..., 5:],'float64'), -1)


In [64]:
intersect_areas.get_shape(), union_areas.get_shape(), iou_scores.get_shape()

(TensorShape([Dimension(1), Dimension(5), Dimension(5), Dimension(2)]),
 TensorShape([Dimension(1), Dimension(5), Dimension(5), Dimension(2)]),
 TensorShape([Dimension(1), Dimension(5), Dimension(5), Dimension(2)]))

In [ ]:

def custom_loss(self, y_true, y_pred):
    mask_shape = tf.shape(y_true)[:4]

    cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(self.grid_w), [self.grid_h]), (1, self.grid_h, self.grid_w, 1, 1)))
    cell_y = tf.transpose(cell_x, (0,2,1,3,4))

    cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [self.batch_size, 1, 1, self.nb_box, 1])

    coord_mask = tf.zeros(mask_shape)
    conf_mask  = tf.zeros(mask_shape)
    class_mask = tf.zeros(mask_shape)

    seen = tf.Variable(0.)
    total_recall = tf.Variable(0.)

    """
    Adjust prediction
    """
    ### adjust x and y      
    pred_box_xy = tf.sigmoid(y_pred[..., :2]) + cell_grid

    ### adjust w and h
    pred_box_wh = tf.exp(y_pred[..., 2:4]) * np.reshape(self.anchors, [1,1,1,self.nb_box,2])

    ### adjust confidence
    pred_box_conf = tf.sigmoid(y_pred[..., 4])

    ### adjust class probabilities
    pred_box_class = y_pred[..., 5:]

    """
    Adjust ground truth
    """
    ### adjust x and y
    true_box_xy = y_true[..., 0:2] # relative position to the containing cell

    ### adjust w and h
    true_box_wh = y_true[..., 2:4] # number of cells accross, horizontally and vertically

    ### adjust confidence
    true_wh_half = true_box_wh / 2.
    true_mins    = true_box_xy - true_wh_half
    true_maxes   = true_box_xy + true_wh_half

    pred_wh_half = pred_box_wh / 2.
    pred_mins    = pred_box_xy - pred_wh_half
    pred_maxes   = pred_box_xy + pred_wh_half       

    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]

    true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
    pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)

    true_box_conf = iou_scores * y_true[..., 4]

    ### adjust class probabilities
    true_box_class = tf.argmax(y_true[..., 5:], -1)

    """
    Determine the masks
    """
    ### coordinate mask: simply the position of the ground truth boxes (the predictors)
    coord_mask = tf.expand_dims(y_true[..., 4], axis=-1) * self.coord_scale

    ### confidence mask: penelize predictors + penalize boxes with low IOU
    # penalize the confidence of the boxes, which have IOU with some ground truth box < 0.6
    true_xy = self.true_boxes[..., 0:2]
    true_wh = self.true_boxes[..., 2:4]

    true_wh_half = true_wh / 2.
    true_mins    = true_xy - true_wh_half
    true_maxes   = true_xy + true_wh_half

    pred_xy = tf.expand_dims(pred_box_xy, 4)
    pred_wh = tf.expand_dims(pred_box_wh, 4)

    pred_wh_half = pred_wh / 2.
    pred_mins    = pred_xy - pred_wh_half
    pred_maxes   = pred_xy + pred_wh_half    

    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]

    true_areas = true_wh[..., 0] * true_wh[..., 1]
    pred_areas = pred_wh[..., 0] * pred_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)

    best_ious = tf.reduce_max(iou_scores, axis=4)
    conf_mask = conf_mask + tf.to_float(best_ious < 0.6) * (1 - y_true[..., 4]) * self.no_object_scale

    # penalize the confidence of the boxes, which are reponsible for corresponding ground truth box
    conf_mask = conf_mask + y_true[..., 4] * self.object_scale

    ### class mask: simply the position of the ground truth boxes (the predictors)
    class_mask = y_true[..., 4] * tf.gather(self.class_wt, true_box_class) * self.class_scale       

    """
    Warm-up training
    """
    no_boxes_mask = tf.to_float(coord_mask < self.coord_scale/2.)
    seen = tf.assign_add(seen, 1.)

    true_box_xy, true_box_wh, coord_mask = tf.cond(tf.less(seen, self.warmup_batches+1), 
                          lambda: [true_box_xy + (0.5 + cell_grid) * no_boxes_mask, 
                                   true_box_wh + tf.ones_like(true_box_wh) * \
                                   np.reshape(self.anchors, [1,1,1,self.nb_box,2]) * \
                                   no_boxes_mask, 
                                   tf.ones_like(coord_mask)],
                          lambda: [true_box_xy, 
                                   true_box_wh,
                                   coord_mask])

    """
    Finalize the loss
    """
    nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
    nb_conf_box  = tf.reduce_sum(tf.to_float(conf_mask  > 0.0))
    nb_class_box = tf.reduce_sum(tf.to_float(class_mask > 0.0))

    loss_xy    = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_wh    = tf.reduce_sum(tf.square(true_box_wh-pred_box_wh)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_conf  = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf) * conf_mask)  / (nb_conf_box  + 1e-6) / 2.
    loss_class = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=true_box_class, logits=pred_box_class)
    loss_class = tf.reduce_sum(loss_class * class_mask) / (nb_class_box + 1e-6)

    loss = tf.cond(tf.less(seen, self.warmup_batches+1), 
                  lambda: loss_xy + loss_wh + loss_conf + loss_class + 10,
                  lambda: loss_xy + loss_wh + loss_conf + loss_class)

    if self.debug:
        nb_true_box = tf.reduce_sum(y_true[..., 4])
        nb_pred_box = tf.reduce_sum(tf.to_float(true_box_conf > 0.5) * tf.to_float(pred_box_conf > 0.3))

        current_recall = nb_pred_box/(nb_true_box + 1e-6)
        total_recall = tf.assign_add(total_recall, current_recall) 

        loss = tf.Print(loss, [loss_xy], message='Loss XY \t', summarize=1000)
        loss = tf.Print(loss, [loss_wh], message='Loss WH \t', summarize=1000)
        loss = tf.Print(loss, [loss_conf], message='Loss Conf \t', summarize=1000)
        loss = tf.Print(loss, [loss_class], message='Loss Class \t', summarize=1000)
        loss = tf.Print(loss, [loss], message='Total Loss \t', summarize=1000)
        loss = tf.Print(loss, [current_recall], message='Current Recall \t', summarize=1000)
        loss = tf.Print(loss, [total_recall/seen], message='Average Recall \t', summarize=1000)

    return loss


In [ ]:
feature_extractor.save('mobile-net-yolo.h5')

In [ ]:
print(feature_extractor.get_output_shape())    
grid_h, grid_w = feature_extractor.get_output_shape()        
features = feature_extractor.extract(input_image)            

# make the object detection layer
output = Conv2D(nb_box * (4 + 1 + nb_class), 
                (1,1), strides=(1,1), 
                padding='same', 
                name='DetectionLayer', 
                kernel_initializer='lecun_normal')(features)
output = Reshape((grid_h, grid_w, nb_box, 4 + 1 + nb_class))(output)
output = Lambda(lambda args: args[0])([output, true_boxes])

model = Model([input_image, true_boxes], output)


# initialize the weights of the detection layer
layer = model.layers[-4]
weights = layer.get_weights()

new_kernel = np.random.normal(size=weights[0].shape)/(grid_h*grid_w)
new_bias   = np.random.normal(size=weights[1].shape)/(grid_h*grid_w)

layer.set_weights([new_kernel, new_bias])

# print a summary of the whole model
model.summary()

In [131]:
len(anchors)//2, 10//2

(62, 5)

In [ ]:
class YOLO(object):
    def __init__(self, backend,
                       input_size, 
                       labels, 
                       max_box_per_image,
                       anchors):

        self.input_size = input_size
        
        self.labels   = list(labels)
        self.nb_class = len(self.labels)
        self.nb_box   = len(anchors)//2
        self.class_wt = np.ones(self.nb_class, dtype='float32')
        self.anchors  = anchors

        self.max_box_per_image = max_box_per_image

        ##########################
        # Make the model
        ##########################

        # make the feature extractor layers
        input_image     = Input(shape=(self.input_size, self.input_size, 3))
        self.true_boxes = Input(shape=(1, 1, 1, max_box_per_image , 4))  

        if backend == 'Inception3':
            self.feature_extractor = Inception3Feature(self.input_size)  
        elif backend == 'SqueezeNet':
            self.feature_extractor = SqueezeNetFeature(self.input_size)        
        elif backend == 'MobileNet':
            self.feature_extractor = MobileNetFeature(self.input_size)
        elif backend == 'Full Yolo':
            self.feature_extractor = FullYoloFeature(self.input_size)
        elif backend == 'Tiny Yolo':
            self.feature_extractor = TinyYoloFeature(self.input_size)
        elif backend == 'VGG16':
            self.feature_extractor = VGG16Feature(self.input_size)
        elif backend == 'ResNet50':
            self.feature_extractor = ResNet50Feature(self.input_size)
        else:
            raise Exception('Architecture not supported! Only support Full Yolo, Tiny Yolo, MobileNet, SqueezeNet, VGG16, ResNet50, and Inception3 at the moment!')

        print(self.feature_extractor.get_output_shape())    
        self.grid_h, self.grid_w = self.feature_extractor.get_output_shape()        
        features = self.feature_extractor.extract(input_image)            

        # make the object detection layer
        output = Conv2D(self.nb_box * (4 + 1 + self.nb_class), 
                        (1,1), strides=(1,1), 
                        padding='same', 
                        name='DetectionLayer', 
                        kernel_initializer='lecun_normal')(features)
        output = Reshape((self.grid_h, self.grid_w, self.nb_box, 4 + 1 + self.nb_class))(output)
        output = Lambda(lambda args: args[0])([output, self.true_boxes])

        self.model = Model([input_image, self.true_boxes], output)

        
        # initialize the weights of the detection layer
        layer = self.model.layers[-4]
        weights = layer.get_weights()

        new_kernel = np.random.normal(size=weights[0].shape)/(self.grid_h*self.grid_w)
        new_bias   = np.random.normal(size=weights[1].shape)/(self.grid_h*self.grid_w)

        layer.set_weights([new_kernel, new_bias])

        # print a summary of the whole model
        self.model.summary()

    def custom_loss(self, y_true, y_pred):
        mask_shape = tf.shape(y_true)[:4]
        
        cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(self.grid_w), [self.grid_h]), (1, self.grid_h, self.grid_w, 1, 1)))
        cell_y = tf.transpose(cell_x, (0,2,1,3,4))

        cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [self.batch_size, 1, 1, self.nb_box, 1])
        
        coord_mask = tf.zeros(mask_shape)
        conf_mask  = tf.zeros(mask_shape)
        class_mask = tf.zeros(mask_shape)
        
        seen = tf.Variable(0.)
        total_recall = tf.Variable(0.)
        
        """
        Adjust prediction
        """
        ### adjust x and y      
        pred_box_xy = tf.sigmoid(y_pred[..., :2]) + cell_grid
        
        ### adjust w and h
        pred_box_wh = tf.exp(y_pred[..., 2:4]) * np.reshape(self.anchors, [1,1,1,self.nb_box,2])
        
        ### adjust confidence
        pred_box_conf = tf.sigmoid(y_pred[..., 4])
        
        ### adjust class probabilities
        pred_box_class = y_pred[..., 5:]
        
        """
        Adjust ground truth
        """
        ### adjust x and y
        true_box_xy = y_true[..., 0:2] # relative position to the containing cell
        
        ### adjust w and h
        true_box_wh = y_true[..., 2:4] # number of cells accross, horizontally and vertically
        
        ### adjust confidence
        true_wh_half = true_box_wh / 2.
        true_mins    = true_box_xy - true_wh_half
        true_maxes   = true_box_xy + true_wh_half
        
        pred_wh_half = pred_box_wh / 2.
        pred_mins    = pred_box_xy - pred_wh_half
        pred_maxes   = pred_box_xy + pred_wh_half       
        
        intersect_mins  = tf.maximum(pred_mins,  true_mins)
        intersect_maxes = tf.minimum(pred_maxes, true_maxes)
        intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
        intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
        
        true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
        pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

        union_areas = pred_areas + true_areas - intersect_areas
        iou_scores  = tf.truediv(intersect_areas, union_areas)
        
        true_box_conf = iou_scores * y_true[..., 4]
        
        ### adjust class probabilities
        true_box_class = tf.argmax(y_true[..., 5:], -1)
        
        """
        Determine the masks
        """
        ### coordinate mask: simply the position of the ground truth boxes (the predictors)
        coord_mask = tf.expand_dims(y_true[..., 4], axis=-1) * self.coord_scale
        
        ### confidence mask: penelize predictors + penalize boxes with low IOU
        # penalize the confidence of the boxes, which have IOU with some ground truth box < 0.6
        true_xy = self.true_boxes[..., 0:2]
        true_wh = self.true_boxes[..., 2:4]
        
        true_wh_half = true_wh / 2.
        true_mins    = true_xy - true_wh_half
        true_maxes   = true_xy + true_wh_half
        
        pred_xy = tf.expand_dims(pred_box_xy, 4)
        pred_wh = tf.expand_dims(pred_box_wh, 4)
        
        pred_wh_half = pred_wh / 2.
        pred_mins    = pred_xy - pred_wh_half
        pred_maxes   = pred_xy + pred_wh_half    
        
        intersect_mins  = tf.maximum(pred_mins,  true_mins)
        intersect_maxes = tf.minimum(pred_maxes, true_maxes)
        intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
        intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
        
        true_areas = true_wh[..., 0] * true_wh[..., 1]
        pred_areas = pred_wh[..., 0] * pred_wh[..., 1]

        union_areas = pred_areas + true_areas - intersect_areas
        iou_scores  = tf.truediv(intersect_areas, union_areas)

        best_ious = tf.reduce_max(iou_scores, axis=4)
        conf_mask = conf_mask + tf.to_float(best_ious < 0.6) * (1 - y_true[..., 4]) * self.no_object_scale
        
        # penalize the confidence of the boxes, which are reponsible for corresponding ground truth box
        conf_mask = conf_mask + y_true[..., 4] * self.object_scale
        
        ### class mask: simply the position of the ground truth boxes (the predictors)
        class_mask = y_true[..., 4] * tf.gather(self.class_wt, true_box_class) * self.class_scale       
        
        """
        Warm-up training
        """
        no_boxes_mask = tf.to_float(coord_mask < self.coord_scale/2.)
        seen = tf.assign_add(seen, 1.)
        
        true_box_xy, true_box_wh, coord_mask = tf.cond(tf.less(seen, self.warmup_batches+1), 
                              lambda: [true_box_xy + (0.5 + cell_grid) * no_boxes_mask, 
                                       true_box_wh + tf.ones_like(true_box_wh) * \
                                       np.reshape(self.anchors, [1,1,1,self.nb_box,2]) * \
                                       no_boxes_mask, 
                                       tf.ones_like(coord_mask)],
                              lambda: [true_box_xy, 
                                       true_box_wh,
                                       coord_mask])
        
        """
        Finalize the loss
        """
        nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
        nb_conf_box  = tf.reduce_sum(tf.to_float(conf_mask  > 0.0))
        nb_class_box = tf.reduce_sum(tf.to_float(class_mask > 0.0))
        
        loss_xy    = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
        loss_wh    = tf.reduce_sum(tf.square(true_box_wh-pred_box_wh)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
        loss_conf  = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf) * conf_mask)  / (nb_conf_box  + 1e-6) / 2.
        loss_class = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=true_box_class, logits=pred_box_class)
        loss_class = tf.reduce_sum(loss_class * class_mask) / (nb_class_box + 1e-6)
        
        loss = tf.cond(tf.less(seen, self.warmup_batches+1), 
                      lambda: loss_xy + loss_wh + loss_conf + loss_class + 10,
                      lambda: loss_xy + loss_wh + loss_conf + loss_class)
        
        if self.debug:
            nb_true_box = tf.reduce_sum(y_true[..., 4])
            nb_pred_box = tf.reduce_sum(tf.to_float(true_box_conf > 0.5) * tf.to_float(pred_box_conf > 0.3))
            
            current_recall = nb_pred_box/(nb_true_box + 1e-6)
            total_recall = tf.assign_add(total_recall, current_recall) 

            loss = tf.Print(loss, [loss_xy], message='Loss XY \t', summarize=1000)
            loss = tf.Print(loss, [loss_wh], message='Loss WH \t', summarize=1000)
            loss = tf.Print(loss, [loss_conf], message='Loss Conf \t', summarize=1000)
            loss = tf.Print(loss, [loss_class], message='Loss Class \t', summarize=1000)
            loss = tf.Print(loss, [loss], message='Total Loss \t', summarize=1000)
            loss = tf.Print(loss, [current_recall], message='Current Recall \t', summarize=1000)
            loss = tf.Print(loss, [total_recall/seen], message='Average Recall \t', summarize=1000)
        
        return loss

    def load_weights(self, weight_path):
        self.model.load_weights(weight_path)

    def train(self, train_imgs,     # the list of images to train the model
                    valid_imgs,     # the list of images used to validate the model
                    train_times,    # the number of time to repeat the training set, often used for small datasets
                    valid_times,    # the number of times to repeat the validation set, often used for small datasets
                    nb_epochs,      # number of epoches
                    learning_rate,  # the learning rate
                    batch_size,     # the size of the batch
                    warmup_epochs,  # number of initial batches to let the model familiarize with the new dataset
                    object_scale,
                    no_object_scale,
                    coord_scale,
                    class_scale,
                    saved_weights_name='best_weights.h5',
                    debug=False):     

        self.batch_size = batch_size

        self.object_scale    = object_scale
        self.no_object_scale = no_object_scale
        self.coord_scale     = coord_scale
        self.class_scale     = class_scale

        self.debug = debug

        ############################################
        # Make train and validation generators
        ############################################

        generator_config = {
            'IMAGE_H'         : self.input_size, 
            'IMAGE_W'         : self.input_size,
            'GRID_H'          : self.grid_h,  
            'GRID_W'          : self.grid_w,
            'BOX'             : self.nb_box,
            'LABELS'          : self.labels,
            'CLASS'           : len(self.labels),
            'ANCHORS'         : self.anchors,
            'BATCH_SIZE'      : self.batch_size,
            'TRUE_BOX_BUFFER' : self.max_box_per_image,
        }    

        train_generator = BatchGenerator(train_imgs, 
                                     generator_config, 
                                     norm=self.feature_extractor.normalize)
        valid_generator = BatchGenerator(valid_imgs, 
                                     generator_config, 
                                     norm=self.feature_extractor.normalize,
                                     jitter=False)   
                                     
        self.warmup_batches  = warmup_epochs * (train_times*len(train_generator) + valid_times*len(valid_generator))   

        ############################################
        # Compile the model
        ############################################

        optimizer = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        self.model.compile(loss=self.custom_loss, optimizer=optimizer)

        ############################################
        # Make a few callbacks
        ############################################

        early_stop = EarlyStopping(monitor='val_loss', 
                           min_delta=0.001, 
                           patience=3, 
                           mode='min', 
                           verbose=1)
        checkpoint = ModelCheckpoint(saved_weights_name, 
                                     monitor='val_loss', 
                                     verbose=1, 
                                     save_best_only=True, 
                                     mode='min', 
                                     period=1)
        tensorboard = TensorBoard(log_dir=os.path.expanduser('~/logs/'), 
                                  histogram_freq=0, 
                                  #write_batch_performance=True,
                                  write_graph=True, 
                                  write_images=False)

        ############################################
        # Start the training process
        ############################################        

        self.model.fit_generator(generator        = train_generator, 
                                 steps_per_epoch  = len(train_generator) * train_times, 
                                 epochs           = warmup_epochs + nb_epochs, 
                                 verbose          = 2 if debug else 1,
                                 validation_data  = valid_generator,
                                 validation_steps = len(valid_generator) * valid_times,
                                 callbacks        = [early_stop, checkpoint, tensorboard], 
                                 workers          = 3,
                                 max_queue_size   = 8)      

        ############################################
        # Compute mAP on the validation set
        ############################################
        average_precisions = self.evaluate(valid_generator)     

        # print evaluation
        for label, average_precision in average_precisions.items():
            print(self.labels[label], '{:.4f}'.format(average_precision))
        print('mAP: {:.4f}'.format(sum(average_precisions.values()) / len(average_precisions)))         

    def evaluate(self, 
                 generator, 
                 iou_threshold=0.3,
                 score_threshold=0.3,
                 max_detections=100,
                 save_path=None):
        """ Evaluate a given dataset using a given model.
        code originally from https://github.com/fizyr/keras-retinanet

        # Arguments
            generator       : The generator that represents the dataset to evaluate.
            model           : The model to evaluate.
            iou_threshold   : The threshold used to consider when a detection is positive or negative.
            score_threshold : The score confidence threshold to use for detections.
            max_detections  : The maximum number of detections to use per image.
            save_path       : The path to save images with visualized detections to.
        # Returns
            A dict mapping class names to mAP scores.
        """    
        # gather all detections and annotations
        all_detections     = [[None for i in range(generator.num_classes())] for j in range(generator.size())]
        all_annotations    = [[None for i in range(generator.num_classes())] for j in range(generator.size())]

        for i in range(generator.size()):
            raw_image = generator.load_image(i)
            raw_height, raw_width, raw_channels = raw_image.shape

            # make the boxes and the labels
            pred_boxes  = self.predict(raw_image)

            
            score = np.array([box.score for box in pred_boxes])
            pred_labels = np.array([box.label for box in pred_boxes])        
            
            if len(pred_boxes) > 0:
                pred_boxes = np.array([[box.xmin*raw_width, box.ymin*raw_height, box.xmax*raw_width, box.ymax*raw_height, box.score] for box in pred_boxes])
            else:
                pred_boxes = np.array([[]])  
            
            # sort the boxes and the labels according to scores
            score_sort = np.argsort(-score)
            pred_labels = pred_labels[score_sort]
            pred_boxes  = pred_boxes[score_sort]
            
            # copy detections to all_detections
            for label in range(generator.num_classes()):
                all_detections[i][label] = pred_boxes[pred_labels == label, :]
                
            annotations = generator.load_annotation(i)
            
            # copy detections to all_annotations
            for label in range(generator.num_classes()):
                all_annotations[i][label] = annotations[annotations[:, 4] == label, :4].copy()
                
        # compute mAP by comparing all detections and all annotations
        average_precisions = {}
        
        for label in range(generator.num_classes()):
            false_positives = np.zeros((0,))
            true_positives  = np.zeros((0,))
            scores          = np.zeros((0,))
            num_annotations = 0.0

            for i in range(generator.size()):
                detections           = all_detections[i][label]
                annotations          = all_annotations[i][label]
                num_annotations     += annotations.shape[0]
                detected_annotations = []

                for d in detections:
                    scores = np.append(scores, d[4])

                    if annotations.shape[0] == 0:
                        false_positives = np.append(false_positives, 1)
                        true_positives  = np.append(true_positives, 0)
                        continue

                    overlaps            = compute_overlap(np.expand_dims(d, axis=0), annotations)
                    assigned_annotation = np.argmax(overlaps, axis=1)
                    max_overlap         = overlaps[0, assigned_annotation]

                    if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations:
                        false_positives = np.append(false_positives, 0)
                        true_positives  = np.append(true_positives, 1)
                        detected_annotations.append(assigned_annotation)
                    else:
                        false_positives = np.append(false_positives, 1)
                        true_positives  = np.append(true_positives, 0)

            # no annotations -> AP for this class is 0 (is this correct?)
            if num_annotations == 0:
                average_precisions[label] = 0
                continue

            # sort by score
            indices         = np.argsort(-scores)
            false_positives = false_positives[indices]
            true_positives  = true_positives[indices]

            # compute false positives and true positives
            false_positives = np.cumsum(false_positives)
            true_positives  = np.cumsum(true_positives)

            # compute recall and precision
            recall    = true_positives / num_annotations
            precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)

            # compute average precision
            average_precision  = compute_ap(recall, precision)  
            average_precisions[label] = average_precision

        return average_precisions    

    def predict(self, image):
        image_h, image_w, _ = image.shape
        image = cv2.resize(image, (self.input_size, self.input_size))
        image = self.feature_extractor.normalize(image)

        input_image = image[:,:,::-1]
        input_image = np.expand_dims(input_image, 0)
        dummy_array = np.zeros((1,1,1,1,self.max_box_per_image,4))

        netout = self.model.predict([input_image, dummy_array])[0]
        boxes  = decode_netout(netout, self.anchors, self.nb_class)

        return boxes